In [1]:
#Testing boto3 aws

import boto3

s3 = boto3.resource('s3')

for bucket in s3.buckets.all():
    print(bucket.name)

crownfloats-test.com
elasticbeanstalk-us-east-1-862738521567
elasticbeanstalk-us-west-2-862738521567
hidden-6ix.com
loginfo-hidden6ix.com
redditpostestimator
www.hidden-6ix.com
